<a href="https://colab.research.google.com/github/sleiph/exploracao-de-dados/blob/main/Pre%C3%A7os_BOVESPA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preço das ações BOVESPA - yahoo finance

In [32]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## lendo os dados do yahoo
from pandas_datareader import data as web

## entrada de dados

In [33]:
# nome do fundo estudado.SA : nome a ser exibido no dataFrame
fundos = {
    'XPCI11.SA': 'XP Credito Imobiliario', 'XPLG11.SA': 'XP LOG', 'XPML11.SA': 'XP MALLS',
    'GRLV11.SA': 'CSHG GR Louveira (HF)', 'HGRU11.SA': 'CSHG Renda Urbana',
    'VILG11.SA': 'Vinci Logistica', 'VINO11.SA': 'Vinci Offices',
    'HCTR11.SA': 'Hectare Ce',
    'KNIP11.SA': 'Kinea Renda Imobiliária'
}

# data pro começo dos dados mm/dd/yyyy
data_inicio = '01/06/2020'

# método de ajuste
ajuste = 'Adj Close'

## populando o dataFrame

In [34]:
# https://medium.com/geleia/an%C3%A1lise-de-a%C3%A7%C3%B5es-com-python-7f6624939fb
df = pd.DataFrame()

for k, v in fundos.items():
  try:
    df[v] = web.get_data_yahoo(k, data_inicio)[ajuste]
  except:
    print("erro no fundo {}".format(v))

## adicionando o índice bovespa
df['IBovespa'] = web.get_data_yahoo('^BVSP', data_inicio)[ajuste]
df['IBovespa'] = df['IBovespa']/1000

## Adicionando as datas como colunas
df.reset_index(inplace = True)
df.rename(columns ={'Date':'Data'},inplace = True)

In [ ]:
# primeiros 5 dias dos dados
df.head()

In [ ]:
# últimos 5 dias dos dados
df[len(df.index)-5:].head(5)

## Gráficos

In [ ]:
# Cotação x tempo - FIIs
tickers = list(df.drop(['Data'], axis = 1).columns)
plt.figure(figsize=(16,6))

for i in tickers:
  plt.plot(df['Data'], df[i])
plt.legend(tickers)
plt.grid()
plt.title("Cotação x tempo", fontsize = 25)
plt.show()

In [ ]:
# médias móveis trimestrais e anuais da XP Credito Imobiliario
plt.figure(figsize=(16,6))
plt.plot(df['Data'], df['XP Credito Imobiliario'].rolling(window = 90).mean())
plt.plot(df['Data'], df['XP Credito Imobiliario'], alpha = 0.8)
plt.plot(df['Data'], df['XP Credito Imobiliario'].rolling(window = 365).mean())
plt.grid()
plt.title('Cotações diárias e médias móveis XP Credito Imobiliario', fontsize = 15)
plt.legend(['Média móvel trimestral','Cotação diária','Média móvel anual'])
plt.show()

In [ ]:
# heatmap das fiis
plt.figure(figsize=(16, 8))
sns.heatmap(df.corr(), annot = True)
plt.show()

In [ ]:
# retorno diário dos fundos
retornos = pd.DataFrame()
for i in tickers:
  retornos[i] = df[i].pct_change()
retornos['Data'] = df['Data']
retornos.describe()

In [ ]:
# pairplot, sei lá pra que serve isso
sns.pairplot(retornos)
plt.show()

In [ ]:
# histplot da XP Credito Imobiliario
sns.histplot(retornos['XP Credito Imobiliario'].dropna())

In [ ]:
# retorno acumulado dos fundos
retornos_soma = pd.DataFrame()

for ticker in tickers:
    retornos_soma[ticker] = (retornos[ticker]+1).cumprod()
retornos_soma['Data'] = retornos['Data']

plt.figure(figsize=(16,6))
plt.plot(retornos_soma['Data'], retornos_soma.drop(['Data'], axis = 1), alpha = 0.9)
plt.legend(tickers)
plt.title("Retorno x tempo", fontsize = 15)
plt.grid()
plt.show()